In [79]:
import os
from dotenv     import load_dotenv

In [80]:
load_dotenv()

True

In [81]:
os.environ['HUGGINGFACE_API_KEY'] = os.getenv('HUGGINGFACE_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

## Search Engine Tools

In [82]:
from langchain_community.tools          import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities  import WikipediaAPIWrapper, ArxivAPIWrapper

## Custom Tools with Wikipedia and Arxiv

In [83]:
wikiWrapperAPI = WikipediaAPIWrapper(
    top_k_results = 1, 
    doc_content_chars_max = 250
)

wiki = WikipediaQueryRun(api_wrapper = wikiWrapperAPI)

In [84]:
wiki.name

'wikipedia'

In [85]:
arxivAPIWrapper = ArxivAPIWrapper(
    top_k_results = 1,
    doc_content_chars_max = 250
)

arxiv = ArxivQueryRun(api_wrapper = arxivAPIWrapper)

In [86]:
arxiv.name

'arxiv'

In [87]:
tools = [wiki, arxiv]

## Custom RAG Tools

In [88]:
from langchain_community.document_loaders       import WebBaseLoader
from langchain_community.vectorstores           import FAISS
from langchain_openai                           import OpenAIEmbeddings
from langchain_text_splitters                   import RecursiveCharacterTextSplitter

In [89]:
webBasedLoader = WebBaseLoader("https://docs.smith.langchain.com/")

In [90]:
docs = webBasedLoader.load()
documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(docs)

In [91]:
vectorStoreDB = FAISS.from_documents(documents, OpenAIEmbeddings())

In [92]:
retriever = vectorStoreDB.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x16826a770>, search_kwargs={})

In [93]:
from langchain.tools.retriever          import create_retriever_tool

retrieverTool = create_retriever_tool(
    retriever,
    "langsmith-search",
    "Search any information about Langsmith"
)

retrieverTool.name

'langsmith-search'

In [94]:
tools = [wiki, retrieverTool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/keshavsaraogi/code/genAI/search-engine-genAI/search_engine_evn/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x10a694160>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x16826a770>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x10aa045e0>, retriever=VectorStoreRetriever(tags=[

In [95]:
from langchain_groq     import ChatGroq

llm = ChatGroq(groq_api_key = groq_api_key, model_name = "Llama3-8b-8192")

## Prompt Template

In [96]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

/Users/keshavsaraogi/code/genAI/search-engine-genAI/search_engine_evn/lib/python3.10/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

## Agents

In [97]:
from langchain.agents   import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [98]:
## Agent Executor

from langchain.agents import AgentExecutor

agentExecutor = AgentExecutor(agent = agent, tools = tools, verbose = True)
agentExecutor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [99]:
agentExecutor.invoke({"input": "Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prot

{'input': 'Tell me about Langsmith',
 'output': "It looks like the tool call yielded some information about LangSmith! Based on the output, LangSmith appears to be a platform for building production-grade Large Language Model (LLM) applications. It seems to provide features such as observability, evaluation, prompt engineering, and deployment for LLM applications.\n\nObservability is a key aspect of LangSmith, allowing developers to closely monitor and evaluate their applications, ship quickly, and with confidence. The platform also provides features for evaluating application performance, getting human feedback, and iterating on prompts.\n\nThe output also mentions LangChain, which seems to be a related project or framework that LangSmith can be used with.\n\nIt seems like LangSmith is a powerful tool for building and deploying LLM applications, and I'm happy to have been able to provide this information!"}

In [100]:
agentExecutor.invoke({"input": "What is Machine Learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine Learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks wit
Invoking: `wikipedia` with `{'query': 'Machine Learning#Applications'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks wit
Invoking: `wikipedia` with `{'query': 'Machine Learning#Applications'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks w

APIError: Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.

In [ ]:
agentExecutor.invoke({"input": "What is the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'paper 1706.03762'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer,
Invoking: `langsmith-search` with `{'query': 'transformer deep learning architecture'}`


Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications.
It al

{'input': 'What is the paper 1706.03762 about?',
 'output': 'No good Wikipedia Search Result was found. It seems that the search query "transformer deep learning architecture Langsmith" did not yield any relevant results. In this case, I will provide a direct response.\n\nThe transformer architecture was introduced in the paper "Attention Is All You Need" by Vaswani et al. in 2017. It\'s a type of recurrent neural network (RNN) that uses self-attention mechanisms to process input sequences. The transformer architecture has gained popularity in natural language processing tasks, such as machine translation, text classification, and language modeling, due to its ability to handle long-range dependencies in input sequences and its parallelization capabilities. Langsmith, on the other hand, is a platform for building production-grade LLM applications, which may use transformer-based architectures in its models.'}